# Stack Overflow Developer Survey 2020 Data Analysis - Part 1

This is a two-part data engineering and analysis of the Stack Overflow Developer Survey 2020, using Python and PostgreSQL. 

The first part is the data engineering part. It consists in data pre-processing, creation of a PostgreSQL database and insertion of the data in said database. This is achieved with pandas, NumPy the psycopg2 driver.

The second part is the data analysis parts. It consists in extracting data from the database, namely for analysis and visualization using Plotly. Part two is available [here](https://so2020devsurvey-part2--ze1598.repl.co/).

There were a total of 64461 respondents for a total of 61 questions. However, less than 30 questions are considered. The main purpose of this analysis is to explore who are the respondents, such as how they use code professionally, their country, academic background, technologies they work with, etc. 

The original dataset can be downloaded from Stack Overflow's website [here](https://insights.stackoverflow.com/survey).

# Imports

In [1]:
import pandas as pd
import numpy as np
from configparser import ConfigParser
import psycopg2
import psycopg2.extras as psql_extras
from typing import Dict, List

# Helper Functions

## Data Transformations

In [2]:
def yearly_comp_calculation(row: pd.Series) -> float:
    """Calculate the yearly compensation taking into account the compensation frequency."""
    if np.isnan(row["ConvertedComp"]):
        return None
    elif row["CompFreq"] == "Weekly":
        return row["ConvertedComp"] * 52
    elif row["CompFreq"] == "Monthly":
        return row["ConvertedComp"] * 12
    elif row["CompFreq"] == "Yearly":
        return row["ConvertedComp"]
    elif row["CompFreq"] == np.nan:
        return row["ConvertedComp"]

In [3]:
def unpivot_delimited_series(
    series: pd.Series,
    delimiter: str
) -> pd.DataFrame:
    """
    Split and unpivot a Series of delimited data, returning a new DataFrame that has two columns reserved for a two-level index that relates the individual values to their original index.
    """

    # First, split each row's data at the specified delimiter,\
    # putting each individual value on its own column.
    # Then, those individual values are unpivoted (stacked) so that the\
    # result is once again one single Series/column
    unpivoted_data = series\
        .apply( lambda series_row: pd.Series(series_row.split(delimiter)) )\
        .stack()

    # Since the above results in a multi-level index, now we need to\
    # reset the Series's index to be able to access those levels as\
    # normal columns later, obtaining a DataFrame
    unpivoted_data = unpivoted_data.reset_index()
    return unpivoted_data

In [4]:
def get_other_colum_names(
    source_df: pd.DataFrame,
    unpivoted_column: str
) -> List[str]:
    """
    Get a list with the name of all the existing columns in the source
    DataFrame, excluding the one that had data with delimiters.
    """

    # Get all the available column names
    other_columns = list(source_df.columns.values)
    # And remove the column that had delimited data
    other_columns.remove(unpivoted_column)
    return other_columns

In [5]:
def unpivot_delimited_data(
    source_df: pd.DataFrame,
    target_column: str,
    delimiter: str
) -> pd.DataFrame:
    """
    Given a DataFrame that has one column with delimited data, split that data and unpivot the individual values into their own rows.
    """

    # Get a DataFrame with the unpivoted data and respective two-level index
    unpivoted_series = unpivot_delimited_series(source_df[target_column], ";")

    # Reset the index of the source DataFrame so we can select the original\
    # index as a normal column
    source_df = source_df.reset_index()

    # Get a list of the source DataFrame columns excluding the one with\
    # delimited data
    source_df_other_columns = get_other_colum_names(source_df, target_column)

    # Choose the columns from the source DataFrame to keep (this includes\
    # a column with the original index, called `index` after the index reset)
    source_data_to_merge = source_df[source_df_other_columns]

    # Merge the columns with non-delimited data to the unpivoted data
    # The merge is based on the original index of the source DataFrame, which\
    # is found as on the first-level index column of the unpivoted data\
    # (`level_0`) and in the old index column of the source data (`index`)
    merged_data = pd.merge(
        unpivoted_series,
        source_data_to_merge,
        how="inner",
        left_on="level_0",
        right_on="index"
    )

    # Rename the column with unpivoted data (it ends up being called 0 (zero))
    merged_data.rename(
        columns={0: target_column},
        inplace=True
    )

    # List with the names of the columns to keep (the only index kept is the\
    # post-reset index of the source DataFrame)
    columns_to_keep = source_df_other_columns + [target_column]
    columns_to_keep.remove("index")

    # Filter out the unwanted columns
    merged_data = merged_data[columns_to_keep]

    return merged_data

## Database Operations

In [6]:
def load_connection_info(
    ini_filename: str
) -> Dict[str, str]:
    
    parser = ConfigParser()
    parser.read(ini_filename)
    # Create a dictionary of the variables stored under the "postgresql" section of the .ini
    conn_info = {param[0]: param[1] for param in parser.items("postgresql")}
    return conn_info

In [7]:
def create_db(
    conn_info: Dict[str, str],
) -> None:
    """
    Create a new PostgreSQL database.
    """
    
    # Connect just to PostgreSQL with the user loaded from the .ini file
    connection_string = f"user={conn_info['user']} password={conn_info['password']}"
    conn = psycopg2.connect(connection_string)
    cur = conn.cursor()

    # "CREATE DATABASE" requires automatic commits
    conn.autocommit = True
    sql_query = f"CREATE DATABASE {conn_info['database']}"

    try:
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        cur.close()
    else:
        # Revert autocommit settings
        conn.autocommit = False

In [8]:
def create_table(
    sql_query: str, 
    conn: psycopg2.extensions.connection, 
    cur: psycopg2.extensions.cursor
) -> None:
    """
    Create a table in a PostgreSQL database.
    """
    
    try:
        # Execute the table creation query
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        conn.rollback()
    else:
        # To take effect, changes need be committed to the database
        conn.commit()

In [9]:
def insert_data(
    query: str,
    conn: psycopg2.extensions.connection,
    cur: psycopg2.extensions.cursor,
    df: pd.DataFrame,
    page_size: int
) -> None:
    
    """
    Insert a pandas DataFrame in a PostgreSQL table, using batch insertion.
    """
    data_tuples = [tuple(row.to_numpy()) for index, row in df.iterrows()]

    try:
        psql_extras.execute_values(cur, query, data_tuples, page_size=page_size)

    except Exception as error:
        print(f"{type(error).__name__}: {error}")
        print("Query:", cur.query)
        conn.rollback()

    else:
        conn.commit()

# Data Pre-Processing

In [10]:
data = pd.read_csv("..\\data\\survey_results_public.csv")
data = data[["Respondent", "MainBranch", "Hobbyist", "Age", "CompFreq", "ConvertedComp", "Country", "DatabaseWorkedWith", "DevType", "EdLevel", "Employment", "Gender", "JobFactors", "LanguageWorkedWith", "NEWCollabToolsWorkedWith", "NEWOvertime", "NEWStuck", "OpSys", "PlatformWorkedWith", "UndergradMajor", "WebframeWorkedWith", "WorkWeekHrs"]]

renamed_columns = {
    "MainBranch": "ProgrammingUsage",
    "DatabaseWorkedWith": "Databases",
    "DevType": "DevRoles",
    "Employment": "EmploymentStatus",
    "LanguageWorkedWith": "ProgLanguages",
    "NEWCollabToolsWorkedWith": "CollabTools",
    "NEWOvertime": "Overtime",
    "NEWStuck": "StuckSolutions",
    "PlatformWorkedWith": "Platforms",
    "UndergradMajor": "UndergradMajor",
    "WebframeWorkedWith": "WebFW",
}
data.rename(columns=renamed_columns, inplace=True)

collab_tools = data[["Respondent", "CollabTools"]]
databases = data[["Respondent", "Databases"]]
dev_roles = data[["Respondent", "DevRoles"]]
job_factors = data[["Respondent", "JobFactors"]]
platforms = data[["Respondent", "Platforms"]]
prog_languages = data[["Respondent", "ProgLanguages"]]
stuck_solutions = data[["Respondent", "StuckSolutions"]]
web_fws = data[["Respondent", "WebFW"]]
respondents = data[["Respondent", "ProgrammingUsage", "Hobbyist", "Age", "CompFreq", "ConvertedComp", "Country", "EdLevel", "EmploymentStatus", "Gender", "Overtime", "OpSys", "UndergradMajor", "WorkWeekHrs"]]

## respondents Table

In [ ]:
respondents.loc[:, "YearlyCompensation"] = respondents.apply(yearly_comp_calculation, axis="columns")
respondents.drop(["CompFreq", "ConvertedComp"], axis="columns", inplace=True)

In [ ]:
respondents["Country"].replace({"nan": None}, inplace=True, regex=True)
respondents.loc[:, "ProgrammingUsage"] = respondents["ProgrammingUsage"].map(
    lambda response: 
        None if (response == np.nan)
        else "Professional developer" if (response == "I am a developer by profession")
        else "Hobbyist developer" if (response == "I code primarily as a hobby")
        else "Worked as a developer before" if (response == "I used to be a developer by profession, but no longer am")
        else "Developer as part of job" if (response == "I am not primarily a developer, but I write code sometimes as part of my work")
        else "Student learning to code" if (response == "I am a student who is learning to code")
        else response
)

In [13]:
respondents["Country"].replace({"nan": None}, inplace=True, regex=True)
respondents.loc[:, "Country"] = respondents["Country"].map(
    lambda response: 
        "United States of America" if (response == "United States")
        else "Russia" if (response == "Russian Federation")
        else response
)

In [14]:
respondents["EdLevel"].replace({"nan": None}, inplace=True, regex=True)
respondents.loc[:, "EdLevel"] = respondents["EdLevel"].map(
    lambda response: 
        "Master's degree" if (response == "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)")
        else "Bachelor's degree" if (response == "Bachelor’s degree (B.A., B.S., B.Eng., etc.)")
        else "High School" if (response == "Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)")
        else "Professional degree" if (response == "Professional degree (JD, MD, etc.)")
        else "Associate degree" if (response == "Associate degree (A.A., A.S., etc.)")
        else "Higher Ed. study w/o degree" if (response == "Some college/university study without earning a degree")
        else "Doctoral degree" if (response == "Other doctoral degree (Ph.D., Ed.D., etc.)")
        else "Elementary school" if (response == "Primary/elementary school")
        else "No formal education" if (response == "I never completed any formal education")
        else response
)

In [15]:
respondents["EmploymentStatus"].replace({"nan": None}, inplace=True, regex=True)
respondents.loc[:, "EmploymentStatus"] = respondents["EmploymentStatus"].map(
    lambda response: 
        "Freelancer/Self-employed" if (response == "Independent contractor, freelancer, or self-employed")
        else response
)

In [16]:
respondents["Gender"].replace({np.nan: None}, inplace=True)
respondents.loc[:, "Gender"] = respondents["Gender"].map(
    lambda response: 
        None if (response == None)
        else "Other" if ("Man" in str(response) and "Woman" in str(response))
        else "Man" if (str(response).startswith("Man"))
        else "Woman" if (str(response).startswith("Woman"))
        else "Other"
)

In [17]:
respondents["Overtime"].replace({"nan": None}, inplace=True, regex=True)
respondents.loc[:, "Overtime"] = respondents["Overtime"].map(
    lambda response: 
        "Often" if (response == "Often: 1-2 days per week or more")
        else "Occasionally" if (response == "Occasionally: 1-2 days per quarter but less than monthly")
        else "Sometimes" if (response == "Sometimes: 1-2 days per month but less than weekly")
        else "Rarely" if (response == "Rarely: 1-2 days per year or less")
        else response
)

In [18]:
respondents["UndergradMajor"].replace({"nan": None}, inplace=True, regex=True)
respondents.loc[:, "UndergradMajor"] = respondents["UndergradMajor"].map(
    lambda response: 
        "CompSci/Eng or SoftEng" if (response == "Computer science, computer engineering, or software engineering")
        else "Another Engineering discipline" if (response == "Another engineering discipline (such as civil, electrical, mechanical, etc.)")
        else "Humanities" if (response == "A humanities discipline (such as literature, history, philosophy, etc.)")
        else "Health Science" if (response == "A health science (such as nursing, pharmacy, radiology, etc.)")
        else "IS/IT or SysAdmin" if (response == "Information systems, information technology, or system administration")
        else "Natural Science" if (response == "A natural science (such as biology, chemistry, physics, etc.)")
        else "Fine Arts/Performing Arts" if (response == "Fine arts or performing arts (such as graphic design, music, studio art, etc.)")
        else "Social Science" if (response == "A social science (such as anthropology, psychology, political science, etc.)")
        else "Business discipline" if (response == "A business discipline (such as accounting, finance, marketing, etc.)")
        else "No major" if (response == "I never declared a major")
        else response
)

In [ ]:
# Replace remaining NaN with None
respondents.replace({np.nan: None}, inplace=True)
respondents.replace({"nan": None}, inplace=True, regex=True)

## collab_tools Table

In [20]:
collab_tools.loc[:, "CollabTools"] = collab_tools.loc[:, "CollabTools"].astype("str")
collab_tools_final = unpivot_delimited_data(collab_tools, "CollabTools", ";")
collab_tools_final.replace({"nan": None}, inplace=True, regex=True)

## databases Table

In [21]:
databases.loc[:, "Databases"] = databases.loc[:, "Databases"].astype("str")
databases_final = unpivot_delimited_data(databases, "Databases", ";")
databases_final.replace({"nan": None}, inplace=True, regex=True)

## dev_roles Table

In [22]:
dev_roles.loc[:, "DevRoles"] = dev_roles.loc[:, "DevRoles"].astype("str")
dev_roles_final = unpivot_delimited_data(dev_roles, "DevRoles", ";")
dev_roles_final["DevRoles"].replace({"nan": None}, inplace=True, regex=True)

dev_roles_final.loc[:, "DevRoles"] = dev_roles_final["DevRoles"].map(
    lambda response: 
        "Desktop applications developer" if (response == "Developer, desktop or enterprise applications")
        else "Full-stack developer" if (response == "Developer, full-stack")
        else "Mobile developer" if (response == "Developer, mobile")
        else "Front-end developer" if (response == "Developer, front-end")
        else "Back-end developer" if (response == "Developer, back-end")
        else "QA developer" if (response == "Developer, QA or test")
        else "Game developer" if (response == "Developer, game or graphics")
        else "Embedded applications developer" if (response == "Developer, embedded applications or devices")
        else "Data engineer" if (response == "Engineer, data")
        else "Data Scientist/ML specialist" if (response == "Data scientist or machine learning specialist")
        else "Site reliability engineer" if (response == "Engineer, site reliability")
        else response
)

## job_factors Table

In [23]:
job_factors.loc[:, "JobFactors"] = job_factors.loc[:, "JobFactors"].astype("str")
job_factors_final = unpivot_delimited_data(job_factors, "JobFactors", ";")
job_factors_final["JobFactors"].replace({"nan": None}, inplace=True, regex=True)

job_factors_final.loc[:, "JobFactors"] = job_factors_final["JobFactors"].map(
    lambda response: 
        "Technical compatibility" if (response == "Languages, frameworks, and other technologies I’d be working with")
        else "Professional development" if (response == "Opportunities for professional development")
        else "Flexible schedule" if (response == "Flex time or a flexible schedule")
        else "Company culture" if (response == "Office environment or company culture")
        else "Diversity of the company" if (response == "Diversity of the company or organization")
        else "Impact of my work" if (response == "How widely used or impactful my work output would be")
        else "Financial performance of the company" if (response == "Financial performance or funding status of the company or organization")
        else "Team/Department fit" if (response == "Specific department or team I’d be working on")
        else response
)

## platforms Table

In [24]:
platforms.loc[:, "Platforms"] = platforms.loc[:, "Platforms"].astype("str")
platforms_final = unpivot_delimited_data(platforms, "Platforms", ";")
platforms_final.replace({"nan": None}, inplace=True, regex=True)

## prog_languages Table

In [25]:
prog_languages.loc[:, "ProgLanguages"] = prog_languages.loc[:, "ProgLanguages"].astype("str")
prog_languages_final = unpivot_delimited_data(prog_languages, "ProgLanguages", ";")
prog_languages_final.replace({"nan": None}, inplace=True, regex=True)

## stuck_solutions Table

In [26]:
stuck_solutions.loc[:, "StuckSolutions"] = stuck_solutions.loc[:, "StuckSolutions"].astype("str")
stuck_solutions_final = unpivot_delimited_data(stuck_solutions, "StuckSolutions", ";")
stuck_solutions_final["StuckSolutions"].replace({"nan": None}, inplace=True, regex=True)

stuck_solutions_final.loc[:, "StuckSolutions"] = stuck_solutions_final["StuckSolutions"].map(
    lambda response: 
        "Visit another developer community" if (response == "Visit another developer community (please name):")
        else response
)

## web_fws Table

In [27]:
web_fws.loc[:, "WebFW"] = web_fws.loc[:, "WebFW"].astype("str")
web_fws_final = unpivot_delimited_data(web_fws, "WebFW", ";")
web_fws_final.replace({"nan": None}, inplace=True, regex=True)

# Database Schema Creation

## Database Creation

In [28]:
conn_info = load_connection_info("db.ini")
create_db(conn_info)

## Connect to sosurvey2020

In [29]:
connection = psycopg2.connect(**conn_info)
cursor = connection.cursor()

## Table Creation

In [30]:
create_sql = """
    CREATE TABLE respondents (
        id INTEGER PRIMARY KEY,
        prog_usage VARCHAR(50),
        hobbyist VARCHAR(3),
        age INTEGER,
        country VARCHAR(50),
        ed_level VARCHAR(50),
        employment_status VARCHAR(50),
        gender VARCHAR(10),
        overtime VARCHAR(20),
        op_sys VARCHAR(20),
        undegrad_major VARCHAR(50),
        work_week_hrs INTEGER,
        yearly_compensation INTEGER
    )
"""
create_table(create_sql, connection, cursor)

In [31]:
create_sql = """
    CREATE TABLE collab_tools (
        id SERIAL PRIMARY KEY,
        collab_tool VARCHAR(50),
        respondent_id INTEGER REFERENCES respondents(id)
    )
"""
create_table(create_sql, connection, cursor)

In [32]:
create_sql = """
    CREATE TABLE databases (
        id SERIAL PRIMARY KEY,
        database VARCHAR(50),
        respondent_id INTEGER REFERENCES respondents(id)
    )
"""
create_table(create_sql, connection, cursor)

In [33]:
create_sql = """
    CREATE TABLE dev_roles (
        id SERIAL PRIMARY KEY,
        dev_role VARCHAR(50),
        respondent_id INTEGER REFERENCES respondents(id)
    )
"""
create_table(create_sql, connection, cursor)

In [34]:
create_sql = """
    CREATE TABLE job_factors (
        id SERIAL PRIMARY KEY,
        job_factor VARCHAR(50),
        respondent_id INTEGER REFERENCES respondents(id)
    )
"""
create_table(create_sql, connection, cursor)

In [35]:
create_sql = """
    CREATE TABLE platforms (
        id SERIAL PRIMARY KEY,
        platform VARCHAR(50),
        respondent_id INTEGER REFERENCES respondents(id)
    )
"""
create_table(create_sql, connection, cursor)

In [36]:
create_sql = """
    CREATE TABLE prog_languages (
        id SERIAL PRIMARY KEY,
        prog_language VARCHAR(50),
        respondent_id INTEGER REFERENCES respondents(id)
    )
"""
create_table(create_sql, connection, cursor)

In [37]:
create_sql = """
    CREATE TABLE stuck_solutions (
        id SERIAL PRIMARY KEY,
        stuck_solution VARCHAR(50),
        respondent_id INTEGER REFERENCES respondents(id)
    )
"""
create_table(create_sql, connection, cursor)

In [38]:
create_sql = """
    CREATE TABLE web_fws (
        id SERIAL PRIMARY KEY,
        web_fw VARCHAR(50),
        respondent_id INTEGER REFERENCES respondents(id)
    )
"""
create_table(create_sql, connection, cursor)

## Data Insertion

In [39]:
insert_query = "INSERT INTO respondents(id, prog_usage, hobbyist, age, country, ed_level, employment_status, gender, overtime, op_sys, undegrad_major, work_week_hrs, yearly_compensation) VALUES %s"
insert_data(insert_query, connection, cursor, respondents, 100)

In [40]:
insert_query = "INSERT INTO collab_tools(respondent_id, collab_tool) VALUES %s"
insert_data(insert_query, connection, cursor, collab_tools_final, 100)

In [41]:
insert_query = "INSERT INTO databases(respondent_id, database) VALUES %s"
insert_data(insert_query, connection, cursor, databases_final, 100)

In [42]:
insert_query = "INSERT INTO dev_roles(respondent_id, dev_role) VALUES %s"
insert_data(insert_query, connection, cursor, dev_roles_final, 100)

In [43]:
insert_query = "INSERT INTO job_factors(respondent_id, job_factor) VALUES %s"
insert_data(insert_query, connection, cursor, job_factors_final, 100)

In [44]:
insert_query = "INSERT INTO platforms(respondent_id, platform) VALUES %s"
insert_data(insert_query, connection, cursor, platforms_final, 100)

In [45]:
insert_query = "INSERT INTO prog_languages(respondent_id, prog_language) VALUES %s"
insert_data(insert_query, connection, cursor, prog_languages_final, 100)

In [46]:
insert_query = "INSERT INTO stuck_solutions(respondent_id, stuck_solution) VALUES %s"
insert_data(insert_query, connection, cursor, stuck_solutions_final, 100)

In [47]:
insert_query = "INSERT INTO web_fws(respondent_id, web_fw) VALUES %s"
insert_data(insert_query, connection, cursor, web_fws_final, 100)

## Close Database Connection

In [48]:
connection.close()
cursor.close()